<a href="https://colab.research.google.com/github/otavioqs16/wine-ia/blob/main/wine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import torch
import numpy as np

In [32]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Net, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.tanh = torch.nn.Tanh()
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        output = self.fc2(relu)
        output = self.tanh(output)
        return output

In [33]:
from sklearn import datasets

In [ ]:
wine = datasets.load_wine()
print(wine)

In [ ]:
dados = wine.data
classes = wine.target
print(dados)
print(classes)

In [36]:
from sklearn.utils import shuffle

In [ ]:
x = dados
y = classes
x, y = shuffle(x, y)
print(x)
print(y)

In [ ]:
for i in range(len(x[1])):
  print(np.max(x[i]))
  max = np.max(x[i])
  if max < 10:
    x[i] /= 10
  elif max < 100:
    x[i] /= 100
  elif max < 1000:
    x[i] /= 1000
  else:
    x[i] /= 10000

for i in range(len(x[1])):
  print(np.max(x[i]))

In [ ]:
training_input = torch.FloatTensor(x[0:142, :])
training_output = torch.FloatTensor(y[0:142])
test_input = torch.FloatTensor(x[142:178, :])
test_output = torch.FloatTensor(y[142:178])
print(test_output.size())
print(test_input.size())

In [40]:
input_size = training_input.size()[1]
hidden_size = 15
model = Net(input_size, hidden_size)
#print(model)
for param in model.parameters():
  #print(param)
  pass

In [ ]:
#Preparando o treinamento
criterion = torch.nn.MSELoss() #Mean Square Error Loss (saida_real - saida_prevista)^2
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

#Avaliar o erro inicial da rede
model.eval() # roda a rede mas sem atualizar os pesos
y_pred = model(test_input)
before_train = criterion(y_pred.squeeze(), test_output)
#[100,] --> [100]
print(f'Erro antes do treinamento: {before_train.item()}')

In [ ]:
#Treinamento da rede
model.train()
epochs = 10000
errors = []
for epoch in range(epochs):
  optimizer.zero_grad()
  #fazer o forward da minha rede
  y_pred = model(training_input)
  #calcula o erro desse treinamento
  loss = criterion(y_pred.squeeze(), training_output)
  errors.append(loss.item())
  if epoch % 1000 == 0:
    print(f'Epoch: {epoch}. Train loss: {loss.item()}')
  #backpropagation
  loss.backward()
  optimizer.step()

In [ ]:
#Avaliar o erro pós treinamento da rede
model.eval() # roda a rede mas sem atualizar os pesos
y_pred = model(test_input)
after_train = criterion(y_pred.squeeze(), test_output)
print(f'Erro pós-treinamento: {after_train.item()}')

In [44]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plotcharts(errors):
    errors = np.array(errors)
    plt.figure(figsize=(12, 5))
    graf02 = plt.subplot(1, 2, 1) # nrows, ncols, index
    graf02.set_title('Errors')
    plt.plot(errors, '-')
    plt.xlabel('Epochs')
    graf03 = plt.subplot(1, 2, 2)
    graf03.set_title('Tests')
    a = plt.plot(test_output.numpy(), 'yo', label='Real')
    plt.setp(a, markersize=10)
    a = plt.plot(y_pred.detach().numpy(), 'b+', label='Predicted')
    plt.setp(a, markersize=10)
    plt.legend(loc=7)
    plt.show()
plotcharts(errors)